In [1]:
import tensorflow as tf
import numpy as np

# If you aren't using gpu, don't excecute below code
# You can read document for below code here
# https://www.tensorflow.org/guide/gpu

# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
# tf.config.experimental.set_memory_growth(gpus[1], True)

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape([x_train.shape[0], x_train.shape[1], x_train.shape[2], 1])
x_test = x_test.reshape([x_test.shape[0], x_test.shape[1], x_test.shape[2], 1])

In [2]:
#If you can't load mnist data form above code, Download mnist.npz and excecute this cell

# from modules.MnistData import MnistData
# data = MnistData('./mnist.npz')

# x_train, y_train = data.x_train, data.y_train
# x_test, y_test = data.x_test, data.y_test

# x_train = x_train.reshape([x_train.shape[0], x_train.shape[1], x_train.shape[2], 1])
# x_test = x_test.reshape([x_test.shape[0], x_test.shape[1], x_test.shape[2], 1])

In [3]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Dropout
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import Dense, Flatten, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Input
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D
import tensorflow as tf

In [4]:
def Build_conv2d(x, channel, stride):
    return (Conv2D(channel, kernel_size=(3, 3), padding='same',
                   kernel_initializer='he_uniform', use_bias=False,
                   strides=stride, activation=None))(x)

In [5]:
def Build_Dense_block(name, x, growthRate):
        shape = x.get_shape().as_list()
        input_channel = shape[3]
        with tf.compat.v1.variable_scope(name) as scope:
            dense = BatchNormalization()(x)
            dense = Activation('relu')
            dense = Build_conv2d(x, growthRate, (1, 1))
            return tf.concat([x, dense], 3)

In [7]:
def Build_transition(name, x):
    shape = x.get_shape().as_list()
    input_channel = shape[3]
    with tf.compat.v1.variable_scope(name) as scope:
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(input_channel, kernel_size=(1, 1),
                   activation=None, strides=(1, 1),
                   use_bias=False)(x)
        return AveragePooling2D([2, 2])(x)

In [15]:
def Build_net(shape, dense_block_num, growthRate):

    channel = 32
    row, col = shape[0], shape[1]
    input_layer = Input(shape=(row, col, 1), name='input')

    x = Build_conv2d(input_layer, channel, (1, 1))

    name_1, name_2, name_3 = 'dense_block_1', 'dense_block_2', 'dense_block_3'

    """
    ====================================
    =========Build First Block==========
    ====================================
    """

    with tf.name_scope(name_1) as scope:

        for i in range(dense_block_num):
            x = Build_Dense_block('dense_layer_{num}'.format(num=i), x, growthRate)

        x = Build_transition('transtion_1', x)

    """
    ====================================
    =========Build Second Block=========
    ====================================
    """

    with tf.name_scope(name_2) as scope:

        for i in range(dense_block_num):
            x = Build_Dense_block('dense_layer_{num}'.format(num=i), x, growthRate)

        x = Build_transition('transtion_2', x)

    """
    ====================================
    =========Build Third Block==========
    ====================================
    """

    with tf.name_scope(name_3) as scope:

        for i in range(dense_block_num):
            x = Build_Dense_block('dense_layer_{num}'.format(num=i), x, growthRate)

        x = Build_transition('transtion_3', x)

    x = BatchNormalization(name='bn_last')(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    out = Dense(10, activation='softmax')(x)
    model = Model(input_layer, out)

    return model

In [16]:
model = Build_net([28, 28], 3, 32)

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 28, 28, 32)   288         input[0][0]                      
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 28, 28, 32)   9216        conv2d_6[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_concat_3 (TensorFlo [(None, 28, 28, 64)] 0           conv2d_6[0][0]                   
                                                                 conv2d_7[0][0]               

In [11]:
epoch = 10
batch_size = 100

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
train_history = model.fit(x_train, y_train, epochs=epoch, batch_size=batch_size,  verbose=1,
                          validation_split=0.1 )

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 11s 209us/sample - loss: 0.1512 - accuracy: 0.9596 - val_loss: 0.2967 - val_accuracy: 0.9063
Epoch 2/10
54000/54000 [==============================] - 10s 177us/sample - loss: 0.0441 - accuracy: 0.9871 - val_loss: 0.1051 - val_accuracy: 0.9663
Epoch 3/10
54000/54000 [==============================] - 10s 177us/sample - loss: 0.0349 - accuracy: 0.9894 - val_loss: 0.1049 - val_accuracy: 0.9672
Epoch 4/10
54000/54000 [==============================] - 10s 180us/sample - loss: 0.0300 - accuracy: 0.9901 - val_loss: 0.0419 - val_accuracy: 0.9883
Epoch 5/10
54000/54000 [==============================] - 10s 182us/sample - loss: 0.0250 - accuracy: 0.9919 - val_loss: 0.0499 - val_accuracy: 0.9855
Epoch 6/10
54000/54000 [==============================] - 10s 183us/sample - loss: 0.0216 - accuracy: 0.9930 - val_loss: 0.0381 - val_accuracy: 0.9883
Epoch 7/10
54000/54000 [=====================

In [18]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 2s - loss: 0.0348 - accuracy: 0.9892


[0.034750204985839084, 0.9892]